# 2/14  1. Введение

ВСПОМНИМ, С КАКИМИ ДАННЫМИ МЫ РАБОТАЕМ

In [ ]:
import pandas as pd
import numpy as np

melb_dfc = pd.read_csv('data/melb_data_fe.csv')
melb_df = melb_dfc.copy()
melb_df.head()

In [ ]:
display(melb_df.info())

`csv`-файл не хранит в себе информацию о типах данных столбцов, поэтому при чтении Pandas автоматически определяет тип данных столбца. Не забывайте об этом, обмениваясь преобразованными данными с вашими коллегами.

Задание 1.1

Преобразуйте столбец `Date` в формат `datetime` и выделите квартал (`quarter`) продажи объектов недвижимости. Найдите второй по популярности квартал продажи. В качестве ответа запишите число объектов, проданных в этом квартале.

In [ ]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'])
melb_df['Date'].dt.quarter.value_counts()


Задание 1.2

Преобразуйте все столбцы, в которых меньше 150 уникальных значений, в тип данных `category`, исключив из преобразования столбцы `Date`, `Rooms`, `Bedroom`, `Bathroom`, `Car`.
В качестве ответа запишите результирующее количество столбцов, которые имеют тип данных `category`.

In [ ]:
unique_list = []
for col in melb_df.columns:
    item = (col, melb_df[col].nunique(), melb_df[col].dtype)
    unique_list.append(item)
unique_counts = pd.DataFrame(
    unique_list, columns=[
        'Colomns', 'Unique', 'Dtype']).sort_values(
            by='Unique', ignore_index=True)
        
display(unique_counts)


In [ ]:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # Список столбцов, которые мы не берём во внимание
max_unique_count = 150 # Задаём максимальное число уникальных категорий
for col in melb_df.columns: # Цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # Проверяем условие
        melb_df[col] = melb_df[col].astype('category') # Преобразуем тип столбца
display(melb_df.info())

# 3/14  2. Сортировка данных в DataFrame

МЕТОД `SORT_VALUES()`

Основные параметры метода `sort_values()`

`by` — имя или список имён столбцов, по значениям которых производится сортировка.  
`axis` — ось, по которой производится сортировка (`0` — строки, `1` — столбцы). По умолчанию сортировка производится по строкам.  
`ascending` — сортировка по **возрастанию** (от меньшего к большему). По умолчанию параметр выставлен на `True`, для сортировки по **убыванию** (от большего к меньшему) необходимо выставить его на `False`.  
`ignore_index` — создаются ли новые индексы в таблице. По умолчанию выставлен на `False` и сохраняет индексы изначальной таблицы.  
`inplace` — производится ли замена исходной таблицы на отсортированную. По умолчанию параметр выставлен на `False`, то есть замены не производится. Чтобы переопределить исходную таблицу на отсортированную, необходимо выставить этот параметр на `True`.

СОРТИРОВКА ПО ЗНАЧЕНИЯМ ОДНОГО СТОЛБЦА

In [ ]:
melb_df.sort_values(by='Price').head(10)

In [ ]:
melb_df.sort_values(by='Date', ascending=False).head(10)

СОРТИРОВКА ПО ЗНАЧЕНИЯМ НЕСКОЛЬКИХ СТОЛБЦОВ

Для сортировки по значениям нескольких столбцов необходимо передать названия этих столбцов в параметр `by` в виде списка. При этом важно обращать внимание на порядок следования столбцов.

Так, например, отсортируем таблицу сначала по возрастанию расстояния от центра города (`Distance`), а затем — по возрастанию цены объекта (`Price`). Для того чтобы вывод был более наглядным, выделим каждую десятую строку из столбцов `Distance` и `Price` результирующей таблицы:

In [ ]:
melb_df.sort_values(
    by=['Distance', 'Price']).loc[::10, ['Distance', 'Price']]

Мы получили таблицу, отсортированную по возрастанию расстояния до центра города. Если встречаются объекты недвижимости, у которых расстояние оказывается одинаковым, то внутри такой группы производится сортировка по цене объекта.

Ради интереса попробуйте поменять порядок следования столбцов в параметре `by` метода `sort_values()` и сравните результат. 

In [ ]:
melb_df.sort_values(
    by=['Price', 'Distance']).loc[::10, ['Distance', 'Price']]

In [ ]:
ddd = {'counts': [np.random.randint(20) for i in range(20)], 'persent':[np.random.randint(20) for i in range(20)]}
ddd1 = pd.DataFrame(ddd)
ddd1.loc[2:19:5,'persent']# Со 2-го по 19-й, но каждый 5-й.

КОМБИНИРОВАНИЕ СОРТИРОВКИ С ФИЛЬТРАЦИЕЙ